# Few Shot Learning

In this notebook, we’ll explore **Few Shot Learning (FSL)** : an important subfield of machine learning where models are trained to generalize effectively from only a few labeled examples.

Traditional deep learning requires large datasets, but in real world scenarios (like medical imaging or rare species identification), collecting many examples is impractical. Few shot learning solves this by leveraging **meta-learning** and **transfer learning** approaches.

## 🎯 Learning Objectives
- Understand the concept and motivation behind Few-Shot Learning
- Learn about **N-way K-shot** terminology
- Explore model architectures used for FSL (e.g., Prototypical Networks, Matching Networks, Siamese Networks)
- Implement a basic **Siamese Network** for few-shot image classification

## 🧩 1. What is Few-Shot Learning?

Few-Shot Learning aims to **learn new tasks or classes from only a few examples per class**. It’s inspired by human learning — we can recognize a new object after seeing it only once or twice.

FSL typically uses **meta-learning**, where the model learns *how to learn* across multiple tasks, enabling it to quickly adapt to new tasks with minimal data.

### 🧮 N-way K-shot Terminology
- **N-way**: Number of classes to classify.
- **K-shot**: Number of examples per class in the support set.
- **Query set**: Unseen examples to test generalization.

**Example:** In a 5-way 1-shot task, the model sees 1 image from each of 5 classes and must classify a new query image into one of them.

## 🧠 2. Common Approaches to Few-Shot Learning

| Approach | Description |
|-----------|--------------|
| **Siamese Networks** | Compare pairs of examples using a distance metric. |
| **Prototypical Networks** | Compute class prototypes (means) in embedding space and classify by distance. |
| **Matching Networks** | Use attention over support examples for classification. |
| **Meta-Learning (MAML)** | Train a model to quickly adapt to new tasks with few updates. |

In [ ]:
# 🧩 3. Siamese Network Example for Few-Shot Learning

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import random

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128*5*5, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
    
    def forward_once(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

## ⚖️ 4. Contrastive Loss Function
The contrastive loss helps the Siamese network learn similarity:

$$ L = (1-Y) * D^2 + (Y) * \max(0, m - D)^2 $$

where:
- **Y = 0** if images are from the same class, **1** otherwise
- **D** is the Euclidean distance between embeddings
- **m** is a margin parameter

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                         (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss

## 🔍 5. Applications of Few-Shot Learning
- 🏥 **Medical diagnosis** with limited labeled scans
- 🐦 **Rare species identification** in ecology
- 🕵️ **Facial recognition** for unseen individuals
- 📱 **Personalized recommendation systems** with few user interactions

## 🧭 6. Summary
- Few-Shot Learning enables learning from few labeled samples.
- Uses meta-learning or similarity-based networks.
- Architectures: Siamese, Prototypical, Matching Networks.
- Crucial for tasks with scarce labeled data.

### ✅ Next Notebook: `06-Zero_Shot_and_One_Shot_Learning.ipynb`
We’ll extend this discussion to **zero-shot** and **one-shot learning**, exploring how models can generalize to completely unseen classes or tasks.